In [1]:
import torch


In [2]:
import torchvision
import torchvision.transforms as transforms

# 불러온 이미지 선처리 하기 위한 단계 (ToTensor : 이미지를 텐서 형태로)
# 전처리 할때 많이 사용한다. 
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

# PIL image (H x W x C) -> Tensor (C x H x W)

trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from torch.utils.data import DataLoader


# 데이터를 한번에 넣어서 학습을 시킬 수 없음, 시간과 메모리를 너무 많이 잡아 먹는다.
# batch_size=4 4개씩 잘라서 분석 진행.
# shuffle은 반복될 때 마다 데이터 재 정립 유무
# drop_last 는 batch_size로 데이터가 나눠 떨어지지 않을 때 그 값들을 버릴 건지
trainloader = DataLoader(trainset, 
                         batch_size=4,
                         shuffle=True, 
                         drop_last=False)

testloader = DataLoader(testset, 
                        batch_size=4,
                        shuffle=False,
                        drop_last=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
#iterator, 데이터를 하나씩 돌면서 탐색한다.
trainiter = iter(trainloader)
images, labels = trainiter.next()

print(len(trainloader))
print(images.shape)
print(labels)

12500
torch.Size([4, 3, 32, 32])
tensor([3, 4, 4, 0])


In [5]:
#parameter setting
src = {'input_size':3*32*32,
       'hidden_size':50,
       'output_size':10,
       'num_epochs':2,
       'batch_size':100,
       'learning_rate':1e-3}

In [6]:
import torch.nn as nn
# nn에 필요한 다양한 함수들을 제공한다.
import torch.nn.functional as F

In [7]:

class TobigsNet(nn.Module):
    def __init__(self, src):
        super(TobigsNet, self).__init__()   
        ## sequential layer
        # 선형->Relu->선형->softmax 
        self.model = nn.Sequential(nn.Linear(src['input_size'], src['hidden_size']),
                     nn.ReLU(),
                     nn.Linear(src['hidden_size'], src['output_size']),
                     nn.Softmax())
        
    
    def forward(self, img):
        # 차원을 바꾼다.

        x = img.view(img.shape[0], -1)
        y = self.model(x)

        return y

In [8]:
# init
model = TobigsNet(src)
# oupput 
y = model(images)

C:\Users\sora\Anaconda3\lib\site-packages\torch\nn\modules\container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [9]:
import torch.optim as optim

# loss 함수로는 likelidhood 사용
criterion = nn.NLLLoss()
# descent gradient 
optimizer = optim.SGD(model.parameters(), 
                      src['learning_rate'], 
                      momentum=0.9)

In [10]:
#진행 표시바
from tqdm import tqdm

for epoch in tqdm(range(src['num_epochs'])):
    #epoch 마다 loss 갱신해야 함으로, current loss =0으로 지정!
    current_loss = 0.0
#     model.train(True)
    
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # loss를 최소화 하는 방향으로, backward gradient descent weight 업데이트!
        
        loss.backward()
        optimizer.step()
        
        # print statistics
        step = i + 1
        current_loss += loss.item()
        
        if step % 1000 == 0 and step != 0:     # print every 1000 mini-batches
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  #  평균 loss를 사용해야 한다.
                  (epoch + 1, src['num_epochs'], step, len(trainloader)//1000 * 1000, current_loss / 1000))
            current_loss = 0.0
        

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\Users\sora\Anaconda3\lib\site-packages\torch\nn\modules\container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/2], Step [1000/12000], Loss: -0.1522
Epoch [1/2], Step [2000/12000], Loss: -0.2144
Epoch [1/2], Step [3000/12000], Loss: -0.2555
Epoch [1/2], Step [4000/12000], Loss: -0.2860
Epoch [1/2], Step [5000/12000], Loss: -0.2876
Epoch [1/2], Step [6000/12000], Loss: -0.2982
Epoch [1/2], Step [7000/12000], Loss: -0.3167
Epoch [1/2], Step [8000/12000], Loss: -0.3174
Epoch [1/2], Step [9000/12000], Loss: -0.3181
Epoch [1/2], Step [10000/12000], Loss: -0.3185
Epoch [1/2], Step [11000/12000], Loss: -0.3190
Epoch [1/2], Step [12000/12000], Loss: -0.3357


 50%|██████████████████████████████████████████                                          | 1/2 [00:40<00:40, 40.11s/it]

Epoch [2/2], Step [1000/12000], Loss: -0.3566
Epoch [2/2], Step [2000/12000], Loss: -0.3568
Epoch [2/2], Step [3000/12000], Loss: -0.3537
Epoch [2/2], Step [4000/12000], Loss: -0.3640
Epoch [2/2], Step [5000/12000], Loss: -0.3673
Epoch [2/2], Step [6000/12000], Loss: -0.3648
Epoch [2/2], Step [7000/12000], Loss: -0.3732
Epoch [2/2], Step [8000/12000], Loss: -0.3722
Epoch [2/2], Step [9000/12000], Loss: -0.3895
Epoch [2/2], Step [10000/12000], Loss: -0.3726
Epoch [2/2], Step [11000/12000], Loss: -0.3747
Epoch [2/2], Step [12000/12000], Loss: -0.3774


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:20<00:00, 40.13s/it]


In [12]:
# Test the Model
correct = 0
total = 0
# test set을 돈다!
for i, data in enumerate(testloader):
    inputs, labels = data
#     images = images.view(-1, 28*28)
    outputs = model(inputs)
    # outputs 확률에서 가장 크게 나온 값을 가져온다!
    _, predicted = torch.max(outputs.data, 1)
    total += labels.shape[0]
    # 만약 두 값이 같으면 sum에 +1 ~!
    
    correct += (predicted == labels).sum()

# 몇 %를 맞췄는지.
print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))

C:\Users\sora\Anaconda3\lib\site-packages\torch\nn\modules\container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Accuracy of the network on the 2500 test images: 39 %
